In [ ]:
%pylab ipympl
ifig = 0

import scipy.integrate
import sys
sys.path.append('../')

import nuclear

reactions = nuclear.network.read_network_file("./networks/cno_cycle") + nuclear.network.read_network_file("./networks/pp_chain")

ifig += 1
nuclear.plot.network_graph(reactions, ifig=ifig)

In [ ]:
rho = 150 # in [g/cm^3]
T = 1.5e7 # in [K]
particles, get_dYdt_epsilon, X_metalicity = nuclear.network.build_network(reactions, dir="./reactions")

print(f"Loaded network using {len(particles):d} isotopes")

X_0 = nuclear.network.initalize_mass_fraction(particles)
Y_0 = nuclear.network.mass_frac_to_molar_abu(particles, X_0)

In [ ]:
t_range = (0, 1e15)
t, Y, status = nuclear.network.solve_network(rho, T, Y_0, get_dYdt_epsilon, t_range)

ifig += 1
nuclear.plot.isotope_evolution(t, Y, particles, ifig=ifig)

## Radial profile test ##

In [ ]:
m = numpy.linspace(0, 1e33, 5)
rho = numpy.linspace(rho, 1e-5, 5)
T = numpy.linspace(T, 1e3, 5)
Y_0 = Y[:, -1][:, None] + 0*rho[None, :]

t_range = (0, 3e9*3e7)
Y, status = nuclear.network.solve_radial_network(rho, T, Y_0, get_dYdt_epsilon, t_range)

ifig += 1
nuclear.plot.isotope_profile(m, Y, particles, ifig=ifig)

In [ ]:
Y_interp = nuclear.network.interpolate_molar_abundance(m, Y)
m = numpy.linspace(m[0], m[-1], 250)
Y = Y_interp(m)

ifig += 1
nuclear.plot.isotope_profile(m, Y, particles, ifig=ifig)